In [1]:
## Импорт библиотек
import pandas as pd
import numpy as np
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense

Загрузка файлов

In [2]:
df_log=pd.read_csv('/content/log.csv',  delimiter = "\t")
df_log.drop(['event_date'], 
            axis=1, inplace=True)

df_log['event_type'] = df_log['event_type'].map({'Добавление в Обращались с номеров':1, "Обращение в службу заботы о клиентах":1, "Выключение IPTV-пакета":1 })

df_log= df_log.astype(float)
df_log.head(3)

,contract_id,event_type
0,36294.0,1.0
1,36294.0,1.0
2,36294.0,1.0


In [3]:
df_named=pd.read_csv('/content/named.csv',  delimiter = "\t")
df_named.drop(['date'], 
            axis=1, inplace=True)
df_named['url'] = df_named['url'].map({'camera.rt.ru	':1, "048056.16.rt.ru":1, "tvip-provision.sampo.ru":1, "webmail.sampo.ru":1, "_sip._udp.048056.16.rt.ru":1  })
df_named = df_named.astype(float)
df_named.head(3)

,url,contract_id
0,1.0,101397.0
1,1.0,179624.0
2,1.0,190335.0


In [4]:
df_type_contract=pd.read_csv('/content/type_contract.csv',  delimiter = ";")
df_type_contract= df_type_contract.astype(float)
df_type_contract.head(3)

,contract_id,day_or_month_contract
0,7780.0,0.0
1,7784.0,0.0
2,7785.0,0.0


In [5]:
df_test = pd.read_csv('/content/test.csv',  delimiter = ";")
df_test= df_test.astype(float)
df_test


,contract_id,blocked
0,275109.0,NaN
1,275285.0,NaN
2,275308.0,NaN
3,275409.0,NaN
4,275628.0,NaN
...,...,...
806,29299.0,NaN
807,12055.0,NaN
808,28564.0,NaN
809,11901.0,NaN


In [6]:
df_train = pd.read_csv('/content/train.csv',  delimiter = ";")
df_train= df_train.astype(float)

In [7]:
df = pd.merge(df_type_contract.append(df_named), df_log, on = ('contract_id')) ## Объединение лог-файлов
df.head()

,contract_id,day_or_month_contract,url,event_type
0,7780.0,0.0,NaN,NaN
1,7780.0,0.0,NaN,1.0
2,7780.0,0.0,NaN,NaN
3,7780.0,0.0,NaN,1.0
4,7780.0,0.0,NaN,1.0


In [8]:
df = df.fillna(0) ## Замена пропущенных значений на 0

In [9]:
df.head()

,contract_id,day_or_month_contract,url,event_type
0,7780.0,0.0,0.0,0.0
1,7780.0,0.0,0.0,1.0
2,7780.0,0.0,0.0,0.0
3,7780.0,0.0,0.0,1.0
4,7780.0,0.0,0.0,1.0


In [10]:
df_1 = pd.concat([df_train, df_test], ignore_index=True) ## Объединеие train и  test 
df_1.head()

,contract_id,blocked
0,7780.0,0.0
1,7785.0,0.0
2,7794.0,0.0
3,7795.0,0.0
4,7798.0,0.0


In [11]:
df_new = df_1.fillna(0) ## замена пропущенных на 0

In [12]:
df_2 = pd.merge(df, df_new, on = ('contract_id')) ## Объединение с большой таблицей

In [13]:
df_2.head(3)

,contract_id,day_or_month_contract,url,event_type,blocked
0,7780.0,0.0,0.0,0.0,0.0
1,7780.0,0.0,0.0,1.0,0.0
2,7780.0,0.0,0.0,0.0,0.0


In [14]:
df_2.drop(['contract_id'], 
            axis=1, inplace=True) ## Убираем столбец с уникальными значениями

In [15]:
df_2.head()

,day_or_month_contract,url,event_type,blocked
0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [16]:
df_2.isnull().sum() ## проверяем на пропуски

day_or_month_contract    0
url                      0
event_type               0
blocked                  0
dtype: int64

In [17]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643442 entries, 0 to 643441
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   day_or_month_contract  643442 non-null  float64
 1   url                    643442 non-null  float64
 2   event_type             643442 non-null  float64
 3   blocked                643442 non-null  float64
dtypes: float64(4)
memory usage: 24.5 MB


In [18]:
y = df_2['blocked'] 

X = df_2.iloc[:, :3]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) ## разбиваем на train и test

In [20]:
y_test

371083    0.0
15232     0.0
286916    0.0
341496    0.0
167474    0.0
         ... 
37475     0.0
1984      0.0
604150    0.0
207969    0.0
109659    0.0
Name: blocked, Length: 128689, dtype: float64

In [21]:
y_test.to_csv('y_test.csv', index = False)

Строим нейронную сеть

In [22]:
model = Sequential([
    Dense(9, activation='relu', input_dim=3),
    Dense(9, activation='relu'),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 36        
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 5)                 50        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 182
Trainable params: 182
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, batch_size=128, epochs=10) ## Обучаем модель

Epoch 1/10
4022/4022 [==============================] - 11s 2ms/step - loss: 0.1827 - accuracy: 0.9588
Epoch 2/10
4022/4022 [==============================] - 10s 2ms/step - loss: 0.1704 - accuracy: 0.9588
Epoch 3/10
4022/4022 [==============================] - 10s 2ms/step - loss: 0.1704 - accuracy: 0.9588
Epoch 4/10
4022/4022 [==============================] - 10s 2ms/step - loss: 0.1704 - accuracy: 0.9588
Epoch 5/10
4022/4022 [==============================] - 10s 2ms/step - loss: 0.1703 - accuracy: 0.9588
Epoch 6/10
4022/4022 [==============================] - 10s 3ms/step - loss: 0.1704 - accuracy: 0.9588
Epoch 7/10
4022/4022 [==============================] - 9s 2ms/step - loss: 0.1703 - accuracy: 0.9588
Epoch 8/10
4022/4022 [==============================] - 11s 3ms/step - loss: 0.1704 - accuracy: 0.9588
Epoch 9/10
4022/4022 [==============================] - 9s 2ms/step - loss: 0.1704 - accuracy: 0.9588
Epoch 10/10
4022/4022 [==============================] - 9s 2ms/step - loss

In [24]:
model.evaluate(X_test, y_test)

4022/4022 [==============================] - 7s 2ms/step - loss: 0.1677 - accuracy: 0.9597


[0.16768796741962433, 0.9597090482711792]

In [25]:
y_pred = model.predict(X_test) ## Делаем предсказание 

4022/4022 [==============================] - 6s 1ms/step


In [26]:
y_pred

array([[0.04362474],
       [0.04362474],
       [0.04362474],
       ...,
       [0.04362474],
       [0.04362474],
       [0.04362474]], dtype=float32)

In [27]:
y_final = (y_pred > 0.5).astype(int).reshape(X_test.shape[0])
y_final

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
# возьмем contract_id из тестовой выборки
ids = df_test['contract_id']
 
# создадим датафрейм из словаря, в котором
# первая пара ключа и значения - это contract_id пассажира, вторая - прогноз "на тесте"
result = pd.DataFrame({'contract_id': [ids], 
                       'blocked': [y_final]})
 
# посмотрим, что получилось
result

,contract_id,blocked
0,0 275109.0 1 275285.0 2 275308....,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [29]:
result.to_csv('result.csv', index = False) ## Сохраним результат

Попробуем LinearRegression

In [30]:
lr = LinearRegression()
lr.fit(X_train,y_train)


LinearRegression()

In [31]:
y_pred_lr = lr.predict(X_test)

In [32]:
y_pred_lr

array([0.03858143, 0.03858143, 0.03858143, ..., 0.03858143, 0.03858143,
       0.03858143])

In [33]:
# возьмем contract_id из тестовой выборки
ids = df_test['contract_id']
 
# создадим датафрейм из словаря, в котором
# первая пара ключа и значения - это contract_id пассажира, вторая - прогноз "на тесте"
result = pd.DataFrame({'contract_id': [ids], 
                       'blocked': [y_pred_lr]})
 
# посмотрим, что получилось
result

,contract_id,blocked
0,0 275109.0 1 275285.0 2 275308....,"[0.03858143337137468, 0.03858143337137468, 0.0..."
